In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('./data/MNIST_data/', one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
trX = trX.reshape(-1, 28, 28, 1)  # reshape trX, -1 means don't consider the number of input images
teX = teX.reshape(-1, 28, 28, 1)  # reshape teX, 28 * 28 * 1 input image

In [4]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [5]:
# define weight initializer
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [6]:
# initialize weights
w = init_weights([3, 3, 1, 32])  # conv kernel is 3*3, input dim is 1, output dim is 32
w2 = init_weights([3, 3, 32, 64])  # input dim is 32 (same as previous layer's output), output dim is 64
w3 = init_weights([3, 3, 64, 128])  # input dim is 64, output dim is 128
# full-connect layer, input dim is 128*4*4, which is convert previous output from 3-dim to 1-dim, output dim is 625
w4 = init_weights([128 * 4 * 4, 625])
w_o = init_weights([625, 10])

In [7]:
# define model
def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    # the first layer of conv, pooling and dropout
    l1a = tf.nn.relu(tf.nn.conv2d(X, w, strides=[1, 1, 1, 1], padding='SAME'))  # l1a shape = (?, 28, 28, 32)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # l1 shape = (?, 14, 14, 32)
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    # the second layer of conv, pooling and dropout
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides=[1, 1, 1, 1], padding='SAME'))  # l2a shape = (?, 14, 14, 64)
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # l2 shape = (?, 7, 7, 64)
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    # the third layer of conv, pooling and dropout
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding='SAME'))  # l3a shape = (?, 7, 7, 128)
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1], strides=[1, 2 ,2, 1], padding='SAME')  # l3 shape = (?, 4, 4, 128)
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])  # reshape to (?, 128*4*4)
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    # full connect layer
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    # output layer
    pyx = tf.matmul(l4, w_o)
    return pyx

In [8]:
#  generate neural networks model
p_keep_conv = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

In [9]:
# define cost function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [10]:
batch_size = 128
test_size = 256
# launch graph in a session
with tf.Session() as sess:
    # initialize all Variables
    tf.global_variables_initializer().run()
    # start training
    for i in range(100):  # 100 epochs
        training_batch = zip(range(0, len(trX), batch_size), range(batch_size, len(trX) + 1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end], p_keep_conv: 0.8, p_keep_hidden: 0.5})
        test_indices = np.arange(len(teX)) # get a test batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0: test_size]
        print(i, np.mean(np.argmax(teY[test_indices], axis=1) == sess.run(predict_op, feed_dict={X: teX[test_indices], 
                                                                                                p_keep_conv: 0.8,
                                                                                                p_keep_hidden: 0.5})))

(0, 0.9453125)
(1, 0.95703125)
(2, 0.984375)
(3, 0.96875)
(4, 0.9765625)
(5, 0.984375)
(6, 0.9765625)
(7, 0.97265625)
(8, 0.99609375)
(9, 0.98828125)
(10, 0.98046875)
(11, 1.0)
(12, 0.984375)
(13, 0.9921875)
(14, 0.984375)
(15, 0.98046875)
(16, 0.9921875)
(17, 0.98828125)
(18, 0.9765625)
(19, 0.9765625)
(20, 0.99609375)
(21, 0.98828125)
(22, 0.9921875)
(23, 0.98828125)
(24, 1.0)
(25, 0.99609375)
(26, 0.98828125)
(27, 0.98828125)
(28, 0.98046875)
(29, 0.9921875)
(30, 0.98046875)
(31, 0.99609375)
(32, 0.9921875)
(33, 1.0)
(34, 0.98828125)
(35, 0.98046875)
(36, 0.99609375)
(37, 0.9921875)
(38, 0.98828125)
(39, 0.9921875)
(40, 0.984375)
(41, 0.97265625)
(42, 0.9921875)
(43, 0.984375)
(44, 0.99609375)
(45, 1.0)
(46, 0.98828125)
(47, 0.98046875)
(48, 0.98828125)
(49, 0.9921875)
(50, 0.9921875)
(51, 0.98046875)
(52, 0.98046875)
(53, 0.99609375)
(54, 0.9921875)
(55, 0.9921875)
(56, 0.96875)
(57, 0.9921875)
(58, 0.98828125)
(59, 0.984375)
(60, 0.98828125)
(61, 0.98828125)
(62, 0.97265625)
(63, 